In [1]:
from PIL import Image
from PIL import ImageOps
import numpy as np
import tarfile
import os
import random

import torch
from torch.utils.data import TensorDataset
from torchvision import transforms

In [2]:
data_base_path = os.path.join(os.pardir, 'data')
data_folder = 'lab-03-data'
tar_path = os.path.join(data_base_path, data_folder + '.tar.gz')
with tarfile.open(tar_path, mode='r:gz') as tar:
    tar.extractall(path=data_base_path)

In [3]:
import gzip
import numpy as np
import matplotlib.pyplot as plt

def extract_data(filename, image_shape, image_number):
    with gzip.open(filename) as bytestream:
        bytestream.read(16)
        buf = bytestream.read(np.prod(image_shape) * image_number)
        data = np.frombuffer(buf, dtype=np.uint8)#.astype(np.float32)
        data = data.reshape(image_number, image_shape[0], image_shape[1])
    return data


def extract_labels(filename, image_number):
    with gzip.open(filename) as bytestream:
        bytestream.read(8)
        buf = bytestream.read(1 * image_number)
        labels = np.frombuffer(buf, dtype=np.uint8).astype(np.uint8)
    return labels

In [4]:
image_shape = (28, 28)
train_set_size = 60000
test_set_size = 10000

data_part2_folder = os.path.join(data_base_path, data_folder, 'part2')

train_images_path = os.path.join(data_part2_folder, 'train-images-idx3-ubyte.gz')
train_labels_path = os.path.join(data_part2_folder, 'train-labels-idx1-ubyte.gz')
test_images_path = os.path.join(data_part2_folder, 't10k-images-idx3-ubyte.gz')
test_labels_path = os.path.join(data_part2_folder, 't10k-labels-idx1-ubyte.gz')

train_images = extract_data(train_images_path, image_shape, train_set_size) 
test_images = extract_data(test_images_path, image_shape, test_set_size)
train_labels = extract_labels(train_labels_path, train_set_size)
test_labels = extract_labels(test_labels_path, test_set_size)


# # Divide by 255.
# train_images = train_images / 255.
# test_images = test_images / 255.

# # Select only 10000
# train_images = train_images[:60000]
# train_labels = train_labels[:60000]
# test_images = test_images[:10000]
# test_labels = test_labels[:10000]

# # Preprocessing
# train_images_p = preprocess_images(train_images[:10000])
# test_images_p = preprocess_images(test_images)

In [5]:
train_original, test_original = list(zip(train_images, train_labels)), list(zip(test_images, test_labels))

In [6]:
print(type(train_original))
print(type(train_original[0]))
print(type(train_original[0][0]))
print(type(train_original[0][1]))
# print(train_original[0])



<class 'list'>
<class 'tuple'>
<class 'numpy.ndarray'>
<class 'numpy.uint8'>


In [7]:
PATH = './my_mnist/'

In [8]:
new_imgs = []
for i in range(92,99,1):
    im = Image.open(PATH + 'IMG_00' + str(i) + '.PNG')
    im = ImageOps.grayscale(im)
    im = ImageOps.invert(im)
    im = im.resize((280,280))
    im_array = np.array(im)

    for i in range(10):
        for j in range(10):
            new_imgs.append((im_array[28*i:28*(i+1),28*j:28*(j+1)], np.uint8(i)))

In [9]:
print(len(new_imgs))
print(new_imgs[0][0].shape)
# new_imgs[0][0].show()
# print(new_imgs[0][0].size)

# for i in range(380,390):
#     Image.fromarray(new_imgs[i][0]).show()

700
(28, 28)


In [10]:
print(type(new_imgs))
print(type(new_imgs[0]))
print(type(new_imgs[0][0]))
print(type(new_imgs[0][1]))
# print(new_imgs[0])




<class 'list'>
<class 'tuple'>
<class 'numpy.ndarray'>
<class 'numpy.uint8'>


In [11]:
new_mnist = []
for i in range(15):
    new_mnist.extend(new_imgs)
random.shuffle(new_mnist)
    
print(len(new_mnist))

10500


In [12]:
aug_mnist = []

aug_mnist.extend(train_original)
aug_mnist.extend(new_mnist)
random.shuffle(aug_mnist)
random.shuffle(aug_mnist)
random.shuffle(aug_mnist)
random.shuffle(aug_mnist)

print(len(aug_mnist))

70500


In [13]:
aug_train = list(map(list,zip(*aug_mnist)))
aug_x_train = np.array(aug_train[0][:])
aug_y_train = np.array(aug_train[1][:])

aug_x_train = list(map(Image.fromarray, aug_x_train))

# aug_x_train, aug_y_train = map(, (aug_x_train, aug_y_train))



# aug_x_train = map(transforms.ToPILImage, aug_x_train)



print(type(aug_x_train))
print(type(aug_y_train))

aug_x_train[52].show()
print(aug_y_train[52])

aug_train = list(zip(aug_x_train, aug_y_train))

print(type(aug_train))
print(type(aug_train[0]))
print(type(aug_train[52][0]))
print(aug_train[52][1])





# print(aug_x_train[0].shape)
# print(aug_y_train.shape)



# print(aug_train[0][0:20])
# print(aug_train[:20][1])

<class 'list'>
<class 'numpy.ndarray'>
7
<class 'list'>
<class 'tuple'>
<class 'PIL.Image.Image'>
7


In [14]:
# train_ds = TensorDataset(aug_x_train, aug_y_train)
torch.save(aug_train, 'my_nmist_list_tuples.pt')

In [15]:
xx = torch.load('my_nmist_list_tuples.pt')

In [16]:
print(type(xx[0][1]))

<class 'numpy.uint8'>


In [17]:
# listA = [1,2,3]
# listB = [4,5,6]
# listC = list(zip(listA,listB))
# listD = list(map(list,zip(*listC)))

# print(listC)
# print(listD)
# print(listD[1][1:3])
# print(listD)

# Operators

In [18]:
PATH2 = './opearators/IMG_0091.PNG'

In [19]:
im2 = Image.open(PATH2)
im2 = ImageOps.grayscale(im2)
im2 = ImageOps.invert(im2)
im2_array = np.array(im2)

In [20]:
im2.show()
im2.size

(1736, 316)

In [21]:
operators_img = []
# for i in range(5):
#     operators_img.append(im2_array[:,347*i:347*(i+1)])

operators_img.append(im2_array[:,:347*(1)-31])
operators_img.append(im2_array[:,347*1:347*(1+1)-31])
operators_img.append(im2_array[:,347*2+15:347*(2+1)-16])
operators_img.append(im2_array[:,347*3+31:347*4])
operators_img.append(im2_array[:,347*4+31:347*5])


In [23]:
print(len(operators_img))
print(operators_img[0].shape)

op_orig = []

for i in range(5):
    op_orig.append((Image.fromarray(operators_img[i]).resize((28,28)), np.uint8(i+10)))
    print(op_orig[i][0].size, op_orig[i][1] )
    

print(type(op_orig))
print(type(op_orig[0]))
print(type(op_orig[0][0]))
print(type(op_orig[0][1]))

5
(316, 316)
(28, 28) 10
(28, 28) 11
(28, 28) 12
(28, 28) 13
(28, 28) 14
<class 'list'>
<class 'tuple'>
<class 'PIL.Image.Image'>
<class 'numpy.uint8'>


In [24]:
new_op_orig = []
for i in range(200):
    new_op_orig.extend(op_orig)
print(len(new_op_orig))

1000


In [25]:
random.shuffle(new_op_orig)
random.shuffle(new_op_orig)
random.shuffle(new_op_orig)
random.shuffle(new_op_orig)
random.shuffle(new_op_orig)

# Handwritten operators

In [26]:
PATH3 = './opearators/IMG_0090.PNG'

In [27]:
new_imgs = []

im = Image.open(PATH3)
im = ImageOps.grayscale(im)
im = ImageOps.invert(im)
im = im.resize((280,280))
im_array = np.array(im)

for i in range(5):
    for j in range(10):
        new_imgs.append((Image.fromarray(im_array[28*i:28*(i+1),28*j:28*(j+1)]), np.uint8(i+10)))
        
for i in range(5,10):
    for j in range(10):
        new_imgs.append((Image.fromarray(im_array[28*i:28*(i+1),28*j:28*(j+1)]), np.uint8(i+5)))
        
print(type(new_imgs))
print(type(new_imgs[0]))
print(type(new_imgs[0][0]))
print(type(new_imgs[0][1]))
print(len(new_imgs))

<class 'list'>
<class 'tuple'>
<class 'PIL.Image.Image'>
<class 'numpy.uint8'>
100


In [28]:
new_op_imgs = []
for i in range(10):
    new_op_imgs.extend(new_imgs)

print(len(new_op_imgs))
print(new_op_imgs[0][0].size)


# print(len(new_imgs))
# print(new_imgs[0][0].shape)
# print(new_imgs[0][0].size)

# for i in range(40,50):
#     Image.fromarray(new_imgs[i][0]).show()
#     print(new_imgs[i][1])

1000
(28, 28)


In [29]:
my_operators = []
my_operators.extend(new_op_orig)
my_operators.extend(new_imgs)

random.shuffle(my_operators)
random.shuffle(my_operators)
random.shuffle(my_operators)
random.shuffle(my_operators)
random.shuffle(my_operators)



In [ ]:
# torch.save(aug_train, 'my_nmist_list_tuples.pt')

In [31]:
new_nmist_plus_op = []
new_nmist_plus_op.extend(my_operators)
new_nmist_plus_op.extend(aug_train)

random.shuffle(new_nmist_plus_op)
random.shuffle(new_nmist_plus_op)
random.shuffle(new_nmist_plus_op)
random.shuffle(new_nmist_plus_op)
random.shuffle(new_nmist_plus_op)


In [34]:
print(len(new_nmist_plus_op))
print(type(new_nmist_plus_op))
print(type(new_nmist_plus_op[0]))
print(type(new_nmist_plus_op[0][0]))
print(type(new_nmist_plus_op[0][1]))




71600
<class 'list'>
<class 'tuple'>
<class 'PIL.Image.Image'>
<class 'numpy.uint8'>


In [35]:
torch.save(new_nmist_plus_op, 'my_nmist_plus_op_list_tuples.pt')